In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Введение в нейронные сети
## Задание по итогам курса:
Задание - Обучите нейронную сеть любой архитектуры на каком-то производственном датасете. Сделайте анализ того, что вам помогло в улучшения работы нейронной сети.
В конце, обязательно подвести вывод. Без этого минус балл.

Датасет жестов взят отсюда:
https://www.kaggle.com/gti-upm/leapgestrecog

Как работать с веб-камерой на google colab
https://stackoverflow.com/questions/54389727/opening-web-camera-in-google-colab




Ссылка на предобученные модели
https://huggingface.co/models

https://huggingface.co/hustvl/yolos-tiny

https://huggingface.co/hustvl/yolos-base

# 1. Загрузка библиотек и данных для обучения модели распознавания жестов

In [15]:
!pip install opencv-python==4.8.1.78

In [16]:
import os
import tqdm
import io
import html
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
#модули для считывания изображений
import cv2
from google.colab.patches import cv2_imshow
import PIL
import PIL.Image

In [18]:
from sklearn.model_selection import train_test_split

from keras import Model
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, UpSampling2D, Reshape, GlobalAveragePooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence, plot_model
from keras.backend import epsilon
from keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf

In [19]:
# модули для web-камеры
# from IPython.display import display, Javascript, Image
# from google.colab.output import eval_js
# from base64 import b64decode, b64encode
# from facenet_pytorch import MTCNN #Модуль для детектирования лица

## 1.1. Импорт датасета жестов

In [20]:
# импорт датасета жестов с kaggle
!pip install -q kaggle
from google.colab import files
files.upload()

# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
# !cp /content/gdrive/MyDrive/kaggle.json .

!ls
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gti-upm/leapgestrecog

# path = '/content/'

KeyboardInterrupt: 

In [ ]:
# !cp -r /content/gdrive/'My Drive'/'PyTorch'/'Photo_for_test' .
#скопировано фото для теста предобученной модели детекции объектов
# img1 = Image.open('/content/Photo_for_test/test1.jpeg')
#img2 = Image.open('/cotent/Photo_for_test/'+'2023-09-03 17-38-00.JPG')
# img3 = Image.open('/cotent/Photo_for_test/2023-08-19 20-43-09.JPG')

In [ ]:
# #Распаковка загруженного датасета
# #!unzip leapgestrecog.zip
# ##Можно так:
# #import shutil
# #shutill.unpack_archive('leapgestrecog.zip', path, 'zip')

# #А можно так:
from zipfile import ZipFile
with ZipFile('/content/leapgestrecog.zip', 'r') as zf:
    zf.extractall()

#папки, где собраны фото жестов
!ls leapGestRecog/

In [ ]:
from pathlib import Path

DATA_ROOT = Path('/content/leapGestRecog/')

 ## 1.2. Изучение входных данных

In [ ]:
#выведу список классов
from torchvision.datasets import ImageFolder
tmp_ds = ImageFolder(DATA_ROOT/'00')
CLASSES_NAMES = tmp_ds.classes
# CLASSES_NAMES.append('00_non_detected')
CLASSES_NAMES

In [ ]:
#оформление спиcка картинок в датафрейм
# gest_df = pd.DataFrame(make_df_from_files(), columns=('img', 'path', 'gest', 'person'))
# gest_df.head(5)

import glob
img_filenames = glob.glob('/content/leapGestRecog/**/**/*.png') #извлекаем имена файлов

# labels = [int(os.path.basename(i).split('_')[2])-1 for i in img_filenames] #извлекаем метки классов


In [ ]:
# всего в датасете 20000 картинок. Оперативка такой объем не тянет. Поэтому выберем из списка файлов 1000 штук рандомным образов
import random
img_filenames = random.sample(img_filenames, 1000)
labels = [int(os.path.basename(i).split('_')[2])-1 for i in img_filenames] #извлекаем метки классов

In [ ]:
print(img_filenames[0])
print("Размерность картинки:", (PIL.Image.open(img_filenames[0])).size)
print("Цветовая кодировка картинки:", PIL.Image.open(img_filenames[0]).mode)

In [ ]:
print(set(labels))
print(type(labels))

In [ ]:
# Функция отрисовки картинки из файлв
def show_image(images, labels, num_pictures, labels_to_compare=None):
    for ind, image in enumerate(images[:num_pictures]):

        img = PIL.Image.open(image)
        plt.imshow(img)
        plt.title(f'Gest class: {CLASSES_NAMES[labels[ind]]}')
        if labels_to_compare != None:
            print(f'True label: {labels_to_compare[i]}')
        plt.show()
        # break

In [ ]:
#посмотрим один из файлов
show_image(img_filenames, labels, 1)

## 1.3. Преобразование входных данных для подачи в модель

In [ ]:
IMAGE_SIZE = 224 #Именно такая рахмерность требуется на вход предобученной модели MobileNetV2

In [ ]:
# import glob
# img_filenames = glob.glob('/content/*.jpg')
# img = PIL.Image.open(img_filenames[0])
# img.size
# # img = img.resize((256, 256))
# # img = np.asarray(img).flatten()
# # plt.imshow(img)

In [ ]:
# привожу каждое изображение к квадратному виду размерности 224*224 и цветовой шкале rgb
batch_images = np.zeros((len(img_filenames), IMAGE_SIZE, IMAGE_SIZE,3), dtype=np.float32)

def prepare_image(image, size=IMAGE_SIZE, color_scale='RGB'):
    """ Принимает на вход считанную из файла картинку и приводит ее к заданному размеру, цветовой шкале и численному типу.
    На выходе получаем картинку в виде массива, который можно подать в модель """

    img = image.resize((size, size))
    img = img.convert(color_scale)
    img = preprocess_input(np.array(img, dtype=np.float32))
    return img

for i, f in enumerate(img_filenames):
    img = PIL.Image.open(img_filenames[i])
    batch_images[i] = prepare_image(img)


In [ ]:
batch_images.shape
#итого у нас батч их 1000 картинок размерностью 224*224 и 3 цветовыми каналами, приведен к типу np.float32

In [ ]:
# посмотрим, как теперь выглядит одна из картинок
plt.imshow(batch_images[10])
plt.title(f'Gest class: {CLASSES_NAMES[labels[10]]}')

In [ ]:
#разбиваю на треин и тест
x_train, x_val, y_train, y_val = train_test_split(batch_images, labels, test_size=0.2, random_state=42)
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

# 2. Обучение модели распознавания жестов

## 2.2. Обучение модели

In [ ]:
#использую предобученную нейросеть MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# замораживаю слои базовой модели , чтобы не обучались
for layer in base_model.layers:
    layer.trainable = False
# добавляю самописные слои для обучения под задачу детекции жестов
x = GlobalAveragePooling2D(name="my_pooling_dence")(base_model.output)
x = Dense(128, activation='relu', name="my_linear_dence")(x)
y_preds = Dense(10, activation='softmax', name="my_clissifier_dence")(x)

# создаю модель
model = Model(inputs=base_model.input, outputs=y_preds)
plot_model(model, show_shapes=True)


In [ ]:
model.compile(optimizer='adam',
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

hist1 = model.fit(x_train, y_train,
                      epochs=10,
                      batch_size=200,
                      validation_data=(x_val, y_val))


In [ ]:
# Оценим модель на тестовых данных, используя "evaluate"
print('\n# Оцениваем на тестовых данных')
results = model.evaluate(x_val, y_val)
print('test loss, test acc:', results)

In [ ]:
# Сгенерируем прогнозы (вероятности - выходные данные последнего слоя)
# на валидационных данных с помощью "predict"
print('\n# Генерируем прогнозы для 3 образцов')
predictions = model.predict(x_val[:3])
print('размерность прогнозов:', predictions.shape)

In [ ]:
#тестирую предсказание класса
dec = model.predict(x_val[10:11])
y_class = np.argmax(dec)
print('Предсказанные вероятности:', np.round(dec, 2))
print('Жест относится к классу:', y_class)

In [ ]:
x_val[10:11].shape

In [ ]:
#функция детектирования жеста
def detect(img, model):
    """ На вход поступает картинка. Происходит предобработка и предсказание
    На выходе получаем класс объекта"""
    x = prepare_image(img)
    inp = np.expand_dims(x, axis=0)
    detect_probs = model.predict(inp[0:1])
    detected_class = np.argmax(detect_probs)
    return detected_class

In [ ]:
#тестирую функцию детекции жестов
img = random.sample(img_filenames, 1)
img = PIL.Image.open(img[0])
y = detect(img, model)
print('Жест относится к классу:', y, CLASSES_NAMES[y])
plt.imshow(img)

In [ ]:
#сохранение модели распознавания жестов
model.save('/content/gest_recognition_tf_model.h5')

# 3. Детекция лица и жеста на реальных данных с веб-камеры

In [ ]:
#Тестирую модель на своих фото
import glob
img_filenames = glob.glob('/content/*.jpg')
img = PIL.Image.open(img_filenames[4])
# img = img.resize((256, 256))
# img = np.asarray(img).flatten()
plt.imshow(img)

In [ ]:
y = detect(img, model)
print('Жест относится к классу:', y, CLASSES_NAMES[y])

In [ ]:
model_loaded = tf.keras.models.load_model('/content/gest_recognition_tf_model.h5')
test_loss, test_acc = model_loaded.evaluate(x_val, y_val, verbose=2)